**SCD1**

In [0]:
data = [(1,"Alice","NewYork", "2010-01-01"),
        (2,"Bob","Chicago", "2010-01-02")]
      
cols =["customer_id", "name", "city", "date"]

df_initial = spark.createDataFrame(data, cols)

df_initial.write.format("delta").mode("overwrite").save("/Volumes/workspace/default/emp/customer_dim")
      

In [0]:
%sql
select * from customer_dim;

In [0]:
update =[(1,"Alice","London", "2011-01-01")]

df_updates = spark.createDataFrame(update, cols)

#df_updates.write.format("delta").mode("append").saveAsTable("customer_dim")

In [0]:
from delta.tables import DeltaTable

dim_path = "/Volumes/workspace/default/emp/customer_dim"

deltaTable = DeltaTable.forPath(spark, dim_path)

deltaTable.alias("tgt").merge(
    df_updates.alias("src"),
    "tgt.customer_id = src.customer_id") \
  .whenMatchedUpdateAll() \
  .whenNotMatchedInsertAll() \
  .execute()


In [0]:
deltaTable.alias("tgt").merge(
    df_updates.alias("src"),
    "tgt.customer_id = src.customer_id") \
  .whenMatchedUpdate(set={
      "name": "src.name",
      "city": "src.city",
      "date": "src.date"})\
  .whenNotMatchedInsert(values = {
      "customer_id": "src.customer_id",
      "name": "src.name",
      "city": "src.city",
      "date": "src.date"}) \
  .execute()

In [0]:
spark.read.format("delta").load(dim_path).show()

**SCD 2**

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

deltaTable.update(
  condition = "True",
  set ={
    "start_date" : "2020-01-01",
    "end_date" : lit(None),
    "Iscurrent" : lit(True)}
)


In [0]:
data1 = [(1,"Alice","NewYork", "2010-01-01",current_date(),lit(None),True),
        (2,"Bob","Chicago", "2010-01-02",current_date(),lit(None),True)]


      
cols1 =["customer_id", "name", "city", "date","start_date","end_date", "Iscurrent"]

df_sc2 = spark.createDataFrame(data1, cols1)

df_sc2.write.format("delta").mode("overwrite").save("/Volumes/workspace/default/emp/customer_dim_scd2")

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType, BooleanType
from pyspark.sql.functions import current_date, lit

data1 = [(1, "Alice", "NewYork", "2010-01-01", "2010-01-01", None, 'Y'),
         (2, "Bob", "Chicago", "2010-01-02", "2010-01-02", None, 'Y')]

schema = StructType([
    StructField("customer_id", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("city", StringType(), True),
    StructField("date", StringType(), True),
    StructField("start_date", StringType(), True),
    StructField("end_date", StringType(), True),
    StructField("Iscurrent", StringType(), True)
])

df_sc2 = spark.createDataFrame(data1, schema)

df_sc2.write.format("delta").mode("overwrite").save("/Volumes/workspace/default/emp/customer_dim_scd2")

In [0]:
deltaTable = DeltaTable.forPath(spark, "/Volumes/workspace/default/emp/customer_dim_scd2")

deltaTable.alias("tgt").merge(
    df_sc2.alias("src"),
    "tgt.customer_id = src.customer_id AND Iscurrent = 'Y'")\
  .whenMatchedUpdate(condition = "src.city > tgt.city",
                     set = {
                         "end_date": current_date(),
                         "Iscurrent": lit(False)
                     })\
  .whenNotMatchedInsert(values = {
      "customer_id": "src.customer_id",
      "name": "src.name",
      "city": "src.city",
      "date": "src.date",
      "start_date": lit("2021-05-01"),
      "end_date": lit(None),
      "Iscurrent": lit('N')
  }).execute()


In [0]:
from pyspark.sql.functions import current_date, lit
from delta.tables import DeltaTable

deltable = DeltaTable.forPath(spark, "/Volumes/workspace/default/emp/customer_dim_scd2")

deltable.alias("tgt").merge(
    df_sc2.alias("src"),
    "tgt.customer_id = src.customer_id AND src.Iscurrent = 'Y'")\
  .whenMatchedUpdate(condition = "src.city > tgt.city",
                     set = {
                         "end_date": current_date(),
                         "Iscurrent": lit(False)
                     })\
  .whenNotMatchedInsert(values = {
      "customer_id": "src.customer_id",
      "name": "src.name",
      "city": "src.city",
      "date": "src.date",
      "start_date": lit("2021-05-01"),
      "end_date": lit(None),
      "Iscurrent": lit('N')
  }).execute()

In [0]:
spark.read.format("delta").load("/Volumes/workspace/default/emp/customer_dim_scd2").show()

In [0]:
update1 =[(4,"John","PEnn", "2011-01-01",None,None,'Y')]

schema1 = StructType([
    StructField("customer_id", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("city", StringType(), True),
    StructField("date", StringType(), True),
    StructField("start_date", StringType(), True),
    StructField("end_date", StringType(), True),
    StructField("Iscurrent", StringType(), True)
])

df_upd_scd2 = spark.createDataFrame(update1, schema1)

In [0]:
from pyspark.sql.functions import current_date, lit
from delta.tables import DeltaTable

deltaTable = DeltaTable.forPath(spark, "/Volumes/workspace/default/emp/customer_dim_scd2")

deltaTable.alias("tgt").merge(
    df_upd_scd2.alias("src"),
    "tgt.customer_id = src.customer_id AND src.Iscurrent = 'Y'")\
  .whenMatchedUpdate(condition = "src.city <> tgt.city",
                     set = {
                         "end_date": current_date(),
                         "Iscurrent": lit('N')
                     })\
  .whenNotMatchedInsert(values = {
      "customer_id": "src.customer_id",
      "name": "src.name",
      "city": "src.city",
      "date": "src.date",
      "start_date": lit("2021-05-01"),
      "end_date": lit(None),
      "Iscurrent": lit('Y')
  }).execute()0

In [0]:
spark.read.format("delta").load("/Volumes/workspace/default/emp/customer_dim_scd2").show()

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from delta.tables import DeltaTable


In [0]:
tgt_claim = DeltaTable.forPath(spark, '/Volumes/dev/bronze/transactions/cptcodes.parquet/')

tgt_cptcodes.alias("tgt").merge(
    

)